In [1]:
import pandas as pd
from pymongo import MongoClient, GEO2D
import pymongo
import json
import datetime
from bson.son import SON
import folium 
import math
import numpy as np
from IPython.display import display

In [2]:
df_20150817 = pd.read_csv('/home/kathrin/Doctorado/bus_rio/Datos_Omnibus/morte_motoqueiro_20150817/20150817.txt', delimiter = ('\t'))

In [3]:
df_20150817_line_317 = df_20150817[(df_20150817['line']=='317')]
#df_20150817_line_317_tarde = df_20150817_line_317[(df_20150817_line_317['timestamp'] > '17-08-2015 22:27:21' ) & (df_20150817_line_317['speed'] == 0 )]
df_20150817_line_317_tarde1 = df_20150817_line_317[(df_20150817_line_317['timestamp'] > '17-08-2015 10:50:21' ) & (df_20150817_line_317['timestamp'] < '17-08-2015 13:10:21' ) ]


In [4]:
onibus317_shapes = pd.read_csv('/home/kathrin/Doctorado/bus_rio/Datos_Omnibus/Onibus/Shapes/gtfs_linha317-shapes.csv')


In [5]:
df_initial_point = onibus317_shapes[(onibus317_shapes['sequencia']==0)]
df_initial_point

,linha,descricao,agencia,sequencia,shape_id,latitude,longitude
0,317,317-CENTRAL X ALVORADA (VIA TUNEL SANTA BARBAR...,Fetranspor,0,17424059,-22.9049,-43.1917
1041,317,317-CENTRAL X ALVORADA (VIA TUNEL SANTA BARBAR...,Fetranspor,0,17589381,-23.0012,-43.3649


In [6]:
#Puntos inicial y final del omnibus 317 Centro -- Alborada
initial_point = (-22.9049, -43.1917)
#Punto inicial del omnibus invertido
inverted_initial_point = (-43.1917, -22.9049)
#Punto final del omnibus 316 Centro
final_point = (-23.0012, -43.3649)
#Punto final del omnibus invertido
inverted_final_point = (-43.3649, -23.0012)

In [9]:
"""Funciones para renderizar el mapa en el html"""
from IPython.display import HTML
 
def inline_map(map):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
    map._build_map()
    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 510px; border: none"></iframe>'.format(srcdoc=map.HTML.replace('"', '&quot;')))
 
def embed_map(map, path="map.html"):
    """
    Embeds a linked iframe to the map into the IPython notebook.
    
    Note: this method will not capture the source of the map into the notebook.
    This method should work for all maps (as long as they use relative urls).
    """
    map.create_map(path=path)
    return HTML('<iframe src="files/{path}" style="width: 100%; height: 510px; border: none"></iframe>'.format(path=path))

In [10]:
onibus317_shapes = pd.read_csv('/home/kathrin/Doctorado/bus_rio/Datos_Omnibus/Onibus/Shapes/gtfs_linha317-shapes.csv')
fmap317=folium.Map(location=[-22.923714, -43.258999], zoom_start=11)
locations317 = []
for row in onibus317_shapes.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    locations317.insert(1, [ row[1]['latitude'], row[1]['longitude'] ])

fmap317.line(locations317)

fmap317.simple_marker(initial_point, popup='Centro')
fmap317.simple_marker(final_point, popup='Alborada')

#Punto inicial
fmap317.circle_marker(location=initial_point, radius=50,line_color='blue',
                 fill_color='#3186cc', fill_opacity=0.2)

#Punto final
fmap317.circle_marker(location=final_point, radius=50,line_color='green',
                 fill_color='#3186cc', fill_opacity=0.2)

embed_map(fmap317)

In [23]:
#esto es para graficar todos los puntos de todos los omnibus 317 que cirsularon ese dia
for row in df_20150817_line_317.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    fmap317.polygon_marker( latlon, fill_color='green', num_sides=4, radius=6)
    

embed_map(fmap317)

In [75]:
onibus317_shapes = pd.read_csv('/home/kathrin/Doctorado/bus_rio/Datos_Omnibus/Onibus/Shapes/gtfs_linha317-shapes.csv')
fmap317_test=folium.Map(location=[-22.923714, -43.258999], zoom_start=11)
locations317 = []
for row in onibus317_shapes.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    locations317.insert(1, [ row[1]['latitude'], row[1]['longitude'] ])

fmap317_test.line(locations317)

fmap317_test.simple_marker(initial_point, popup='Centro')
fmap317_test.simple_marker(final_point, popup='Alborada')

#Punto inicial
fmap317_test.circle_marker(location=initial_point, radius=50,line_color='blue',
                 fill_color='#3186cc', fill_opacity=0.2)

#Punto final
fmap317_test.circle_marker(location=final_point, radius=50,line_color='green',
                 fill_color='#3186cc', fill_opacity=0.2)

for row in df_20150817_line_317_tarde1.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    fmap317_test.polygon_marker( latlon, fill_color='green', num_sides=4, radius=6)



embed_map(fmap317_test)

In [15]:
#Estableciendo la conexion con la base de datos morte_motoquiro en MongoDB
db = MongoClient().morte_motoquiro_event

In [11]:
#funcion que dado un el bus_id de un omnibus y un punto inicial/final devuelve los puntos mas cercanos a ese punto. 
# A esa función se le debería pasar también la BD, pero ahora no se va a hacer asi
def near_points_to_origin (bus_id, inverted_origin_point):
    near_initial_points_array = []
    #la funcion $within recibe las coordenadas en orden inverso (primero longitud y después latitud)
    for row in db.dados20150817_line_317.find({"latlon": {"$within": {"$center": [inverted_origin_point, 0.0115499543]}}, "bus_id": bus_id}).sort("timestamp",pymongo.DESCENDING):
        near_initial_points_array.insert(0, [row[u'timestamp'], row[u'latitude'], row[u'longitude'], row[u'speed']])
    #convert the array in panda dataframe 
    sorted_near_initial_points = pd.DataFrame(near_initial_points_array, columns=['timestamp', 'latitude', 'longitude', 'speed'])
    return sorted_near_initial_points

In [13]:
sorted_near_initial_points = near_points_to_origin ('C41405', inverted_initial_point)
sorted_near_initial_points

,timestamp,latitude,longitude,speed
0,17-08-2015 10:45:56,-22.900803,-43.181801,15.37
1,17-08-2015 10:47:08,-22.902279,-43.183056,11.67
2,17-08-2015 10:48:17,-22.904301,-43.189445,41.86
3,17-08-2015 10:48:52,-22.904623,-43.190186,1.11
4,17-08-2015 10:50:02,-22.905161,-43.192055,0.74
5,17-08-2015 10:51:12,-22.905624,-43.193359,14.08
6,17-08-2015 10:52:22,-22.905584,-43.196877,2.96
7,17-08-2015 10:52:57,-22.905588,-43.196854,0.37
8,17-08-2015 10:53:29,-22.905584,-43.196735,0.37
9,17-08-2015 10:58:56,-22.905600,-43.196781,0.00


In [14]:
sorted_near_final_points = near_points_to_origin ('C41405', inverted_final_point)
sorted_near_final_points

,timestamp,latitude,longitude,speed
0,17-08-2015 08:17:07,-23.010530,-43.363117,50.93
1,17-08-2015 08:18:17,-23.008617,-43.366085,0.37
2,17-08-2015 08:18:52,-23.006783,-43.366085,42.78
3,17-08-2015 08:20:02,-23.000648,-43.362877,21.48
4,17-08-2015 08:21:12,-23.000240,-43.358501,0.56
5,17-08-2015 08:21:48,-23.000254,-43.358475,2.04
6,17-08-2015 08:22:57,-23.000046,-43.355118,28.15
7,17-08-2015 08:24:07,-22.999460,-43.358959,24.82
8,17-08-2015 08:25:18,-22.997169,-43.365425,43.89
9,17-08-2015 08:25:52,-22.999584,-43.368980,44.45


In [27]:
#Function to compute the bus_ids circulating during a day 20150208 of bus line 438
def bus_id_list ():
    bus_id_20150817_317 = db.dados20150817_line_317.distinct("bus_id")
    return bus_id_20150817_317

In [28]:
#llamada a la funcion bus_id_list
busId_list = bus_id_list()
busId_list

[u'C41405', u'C41406', u'C41401', u'C41408']

In [29]:
#Funcion para meter los bus_Id del dataframe para un array
def get_busId_list():
    array_busId = []
    for bus in busId_list:
        strbus = str(bus)
        array_busId.insert(len(array_busId), strbus)
    return array_busId

In [30]:
array_busId = get_busId_list()
array_busId

['C41405', 'C41406', 'C41401', 'C41408']

In [23]:
df_20150817_line_317_C41405 = df_20150817_line_317[(df_20150817_line_317['bus_id']=='C41405')]
df_20150817_line_317_C41405 
#para seleccionar solo algunas columnas del dataframe, en este caso la columna 1, la 4 y la 5
important_columns = df_20150817_line_317_C41405.iloc[:,[0,3,4]]
important_columns 

,timestamp,latitude,longitude
776381,17-08-2015 06:54:52,-22.938576,-43.183338
780770,17-08-2015 06:56:02,-22.943159,-43.183239
784536,17-08-2015 06:57:12,-22.945505,-43.183838
786589,17-08-2015 06:57:47,-22.949413,-43.184383
790017,17-08-2015 06:58:57,-22.949526,-43.187683
794345,17-08-2015 07:00:07,-22.949671,-43.189934
797619,17-08-2015 07:01:17,-22.950451,-43.192410
798671,17-08-2015 07:01:52,-22.950867,-43.193596
803142,17-08-2015 07:03:02,-22.952301,-43.196175
806531,17-08-2015 07:04:12,-22.955200,-43.197857


In [54]:
myvector = important_columns.values.tolist()


In [29]:
initial_point

(-22.9049, -43.1917)

In [30]:
final_point

(-23.0012, -43.3649)

In [41]:
lat_initial_point = initial_point[0]
long_initial_point = initial_point[1]

In [45]:
lat_final_point = final_point[0]
long_final_point = final_point[1]

In [50]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return (c * r)*1000 # return the distance in m

 

In [51]:
distance = haversine(long_initial_point, lat_initial_point, long_final_point, lat_final_point)
distance

20716.223790977547

In [61]:
"""
funcion que dado un vector de lat, long y un punto A(lat,long), me devuelve un vector con la distancia de cada par al punto A
 """
def vector_distancia(vector, point):
    vector_distance = []
    lat2 = point[0]
    long2 = point[1]
    for item in vector:
        lat1 = item[1]
        long1 = item[2]
        distance = haversine(long2, lat2, long1, lat1)
        vector_distance.insert(0, [item[0], distance])
    df_vector_distance = pd.DataFrame(vector_distance, columns=['timestamp', 'distance'])
    return df_vector_distance        
        
    

In [62]:
dist_to_ini = vector_distancia(myvector, initial_point)
dist_to_ini

,timestamp,distance
0,17-08-2015 23:33:41,7520.325036
1,17-08-2015 23:31:15,7538.620241
2,17-08-2015 23:04:09,7504.677062
3,17-08-2015 23:02:59,7502.207114
4,17-08-2015 22:34:51,7481.343215
5,17-08-2015 22:30:26,7469.001159
6,17-08-2015 22:30:06,7469.631141
7,17-08-2015 22:28:56,7489.427379
8,17-08-2015 22:28:21,7498.990886
9,17-08-2015 22:27:08,7503.769733


In [66]:
nearA = dist_to_ini[(dist_to_ini['distance']< 1000)]
nearA

,timestamp,distance
167,17-08-2015 13:12:03,985.391354
168,17-08-2015 13:10:53,736.377152
169,17-08-2015 13:10:18,714.938513
170,17-08-2015 13:08:33,524.813760
171,17-08-2015 13:07:58,523.032854
172,17-08-2015 13:05:03,524.465902
173,17-08-2015 13:02:41,529.295678
174,17-08-2015 13:01:20,527.670034
175,17-08-2015 13:00:10,530.005564
176,17-08-2015 12:59:00,528.865680


In [16]:
mynearfinal = near_points_to_origin ('C41405', inverted_final_point)
mynearfinal

,timestamp,latitude,longitude,speed
0,17-08-2015 08:17:07,-23.010530,-43.363117,50.93
1,17-08-2015 08:18:17,-23.008617,-43.366085,0.37
2,17-08-2015 08:18:52,-23.006783,-43.366085,42.78
3,17-08-2015 08:20:02,-23.000648,-43.362877,21.48
4,17-08-2015 08:21:12,-23.000240,-43.358501,0.56
5,17-08-2015 08:21:48,-23.000254,-43.358475,2.04
6,17-08-2015 08:22:57,-23.000046,-43.355118,28.15
7,17-08-2015 08:24:07,-22.999460,-43.358959,24.82
8,17-08-2015 08:25:18,-22.997169,-43.365425,43.89
9,17-08-2015 08:25:52,-22.999584,-43.368980,44.45


In [20]:
#Array de timestamp puntos near punto incial (Leblon)
initial_points_timestamp = sorted_near_initial_points['timestamp']
initial_points_timestamp

0     17-08-2015 10:45:56
1     17-08-2015 10:47:08
2     17-08-2015 10:48:17
3     17-08-2015 10:48:52
4     17-08-2015 10:50:02
5     17-08-2015 10:51:12
6     17-08-2015 10:52:22
7     17-08-2015 10:52:57
8     17-08-2015 10:53:29
9     17-08-2015 10:58:56
10    17-08-2015 11:00:06
11    17-08-2015 11:01:16
12    17-08-2015 11:28:57
13    17-08-2015 11:30:07
14    17-08-2015 11:31:17
15    17-08-2015 11:58:58
16    17-08-2015 12:00:08
17    17-08-2015 12:01:18
18    17-08-2015 12:28:59
19    17-08-2015 12:30:09
20    17-08-2015 12:31:19
21    17-08-2015 12:59:00
22    17-08-2015 13:00:10
23    17-08-2015 13:01:20
24    17-08-2015 13:02:41
25    17-08-2015 13:05:03
26    17-08-2015 13:07:58
27    17-08-2015 13:08:33
28    17-08-2015 13:10:18
29    17-08-2015 13:10:53
30    17-08-2015 13:12:03
Name: timestamp, dtype: object

In [23]:
a = bus_departure_hour(initial_points_timestamp)
a

[datetime.datetime(2015, 8, 17, 13, 12, 3)]

In [25]:
final_points_timestamp = sorted_near_final_points['timestamp']


In [21]:
def bus_departure_hour (initial_points_timestamp_list):
    list_departure_hours = []
    i = 0
    if len(initial_points_timestamp_list) >1:
        for i in range(0, len(initial_points_timestamp_list)-1):
            resto = datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S")-datetime.datetime.strptime(initial_points_timestamp_list[i], "%d-%m-%Y %H:%M:%S")
            resto_total = resto.total_seconds()/60
            #si la diferencia entre los timestamp del omnibus en el putno supera los 40 minutos es q dio un viaje y regreso, sino lo supera es que se mantuvo ese tiempo esperando en el punto
            if (resto_total > 40):
                 list_departure_hours.insert(len( list_departure_hours),datetime.datetime.strptime(initial_points_timestamp_list[i], "%d-%m-%Y %H:%M:%S"))
        #siempre es necesario insertar el ultimo valor de la lista de "initial_points_timestamp" en la lista de  list_departure_hours 
        list_departure_hours.insert(len( list_departure_hours),datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S"))
    elif len(initial_points_timestamp_list) == 1:
        list_departure_hours.insert(len( list_departure_hours),datetime.datetime.strptime(initial_points_timestamp_list[0], "%d-%m-%Y %H:%M:%S"))
    else:
        list_departure_hours = []
    return list_departure_hours

In [22]:
#Funcion que retorna el arreglo con las horas de llegada del omnibus al punto inicial/final del trayecto
def bus_arrive_hour (initial_points_timestamp_list):
    list_arrive_hours = []
    #siempre es necesario insertar el ultimo valor de la lista de "initial_points_timestamp" en la lista de leblon_departure_hours 
    if len(initial_points_timestamp_list) >1:
        list_arrive_hours.insert(len(list_arrive_hours),datetime.datetime.strptime(initial_points_timestamp_list[0], "%d-%m-%Y %H:%M:%S"))
        for i in range(0,  len(initial_points_timestamp_list)-1):
            resto = datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S")-datetime.datetime.strptime(initial_points_timestamp_list[i], "%d-%m-%Y %H:%M:%S")
            resto_total = resto.total_seconds()/60
            if (resto_total > 40):
                list_arrive_hours.insert(len(list_arrive_hours),datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S"))
    elif (len(initial_points_timestamp_list) == 1):
        list_arrive_hours.insert(len(list_arrive_hours),datetime.datetime.strptime(initial_points_timestamp_list[0], "%d-%m-%Y %H:%M:%S"))
    else:
        list_arrive_hours = []        
    return list_arrive_hours

In [64]:
#Para convertir de formato timedelta para horas y minutos
def days_hours_minutes(travel_time):
    hours = travel_time.seconds//3600 
    minutes = (travel_time.seconds//60)%60
    #print '%d hours, %d minutes' % (hours,minutes)
    if minutes < 10:
        minutes = str(0)+str(minutes)
    return str(hours) + ":"+ str(minutes)

In [65]:
#Funcion que crea la tabla de viajes con los tiempos de viaje
def create_travel_table (name_A, arrive_array_A, departure_array_A, name_B, arrive_array_B, departure_array_B, bus_Id, bus_line):
    
    my_time_table = []   
    num_travel = 0    
    i = 0 #Cursor para moverme por el departure_array_A(arreglo q contiene las horas de salida del punto incial)   
    j = 0 #Cursor para moverme por el departure_array_B (arreglo q contiene las horas de salida del punto final)
   
    while (i<len(departure_array_A) and j< len(departure_array_B)):
        if (departure_array_A[i] < departure_array_B[j]):
            if(i+1 == len(departure_array_A)):
                direction = name_A + "_to_" + name_B
                departure_hour = departure_array_A[i]
                i = i +1
                arrive_hour = arrive_array_B[j]
                num_travel = num_travel +1
                travel_time = days_hours_minutes(arrive_hour - departure_hour)          
                my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
         
            elif ((i+1 < len(departure_array_A)) and (departure_array_B[j] < departure_array_A[i+1])):
                direction = name_A + "_to_" + name_B
                departure_hour = departure_array_A[i]
                i = i +1
                arrive_hour = arrive_array_B[j]
                num_travel = num_travel +1
                travel_time = days_hours_minutes(arrive_hour - departure_hour)          
                my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
            else: 
                i = i +1
            
        elif(departure_array_B[j] < departure_array_A[i]):
            if (j+1 == len(departure_array_B)):
                direction = name_B + "_to_" + name_A
                departure_hour = departure_array_B[j]
                j = j +1
                arrive_hour = arrive_array_A[i]
                num_travel = num_travel +1
                travel_time = days_hours_minutes(arrive_hour - departure_hour)          
                my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
            
            elif ((j+1 < len(departure_array_B)) and (departure_array_A[i] < departure_array_B[j+1])):
                direction = name_B + "_to_" + name_A
                departure_hour = departure_array_B[j]
                j = j +1
                arrive_hour = arrive_array_A[i]
                num_travel = num_travel +1
                travel_time = days_hours_minutes(arrive_hour - departure_hour)          
                my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
            else:
                j = j +1                
            
    #Construir el dataframe pasandole el array con todos los datos de los viajes   
    mydf = pd.DataFrame(my_time_table, columns=['Line', 'Bus_Id','Direction','Departure_Hour', 'Arrive_Hour', 'Travel_Time'])       
    return mydf  


In [66]:
#Funcion que crea la tabla de tiempo de viaje para cada bus_id de los que transitaron el el dia
def all_travel_table (line, punto_ini_coord_invertidas, name_punto_inicio, punto_fin_coord_invertidas, name_punto_final, array_busId):
    new_df = pd.DataFrame(columns=['Line', 'Bus_Id','Direction','Departure_Hour', 'Arrive_Hour', 'Travel_Time'])  
    for busId in array_busId:
        sorted_near_initial_points1 = near_points_to_origin(busId, punto_ini_coord_invertidas)
        sorted_near_final_points1 = near_points_to_origin(busId, punto_fin_coord_invertidas)
        initial_points_timestamp1 = sorted_near_initial_points1 ['timestamp']
        final_points_timestamp1 = sorted_near_final_points1 ['timestamp']
        #chequear si la lista initial_points_timestamp es diferente de vacio
        if len(initial_points_timestamp1) and len(final_points_timestamp1):
            vilaIsabel_departure_hours1 = bus_departure_hour(initial_points_timestamp1)
            first_point_departure_hour = bus_departure_hour (initial_points_timestamp1)
            last_point_departure_hour = bus_departure_hour (final_points_timestamp1)
            first_point_arrive_hour = bus_arrive_hour(initial_points_timestamp1)
            last_point_arrive_hour = bus_arrive_hour(final_points_timestamp1)
            my_time_table = create_travel_table(name_punto_inicio, first_point_arrive_hour, first_point_departure_hour, name_punto_final, last_point_arrive_hour, last_point_departure_hour, busId, line)
            new_df = new_df.append(my_time_table, ignore_index=True)
    return new_df

In [69]:
query = all_travel_table ('317', inverted_initial_point, 'Alborada', inverted_final_point, 'Central', array_busId)
query

,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time
0,317,C41405,Central_to_Alborada,2015-08-17 08:49:48,2015-08-17 10:45:56,1:56
1,317,C41405,Alborada_to_Central,2015-08-17 13:12:03,2015-08-17 14:20:17,1:08
2,317,C41406,Alborada_to_Central,2015-08-17 07:59:03,2015-08-17 09:27:51,1:28
3,317,C41406,Central_to_Alborada,2015-08-17 09:52:02,2015-08-17 11:15:19,1:23
4,317,C41406,Alborada_to_Central,2015-08-17 14:52:22,2015-08-17 16:37:11,1:44
5,317,C41406,Central_to_Alborada,2015-08-17 17:16:56,2015-08-17 19:02:22,1:45
6,317,C41406,Alborada_to_Central,2015-08-17 19:15:48,2015-08-17 20:59:13,1:43
7,317,C41406,Central_to_Alborada,2015-08-17 21:17:56,2015-08-17 22:27:21,1:09
8,317,C41401,Alborada_to_Central,2015-08-17 08:56:56,2015-08-17 10:30:07,1:33
9,317,C41401,Central_to_Alborada,2015-08-17 11:10:03,2015-08-17 12:49:06,1:39


In [40]:
direction_central_Alborada = query[(query['Direction'] == 'Central_to_Alborada')]
direction_central_Alborada_sorted = direction_central_Alborada.sort(["Departure_Hour"])
direction_central_Alborada_sorted


,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time
0,317,C41405,Central_to_Alborada,2015-08-17 08:41:02,2015-08-17 10:48:52,2:07
2,317,C41406,Central_to_Alborada,2015-08-17 09:49:07,2015-08-17 11:18:57,1:29
14,317,C41408,Central_to_Alborada,2015-08-17 10:21:50,2015-08-17 11:51:55,1:30
8,317,C41401,Central_to_Alborada,2015-08-17 11:05:05,2015-08-17 12:50:51,1:45
16,317,C41408,Central_to_Alborada,2015-08-17 14:00:59,2015-08-17 15:24:23,1:23
10,317,C41401,Central_to_Alborada,2015-08-17 14:57:56,2015-08-17 16:25:10,1:27
4,317,C41406,Central_to_Alborada,2015-08-17 17:12:18,2015-08-17 19:04:42,1:52
18,317,C41408,Central_to_Alborada,2015-08-17 17:56:23,2015-08-17 19:31:07,1:34
12,317,C41401,Central_to_Alborada,2015-08-17 19:53:11,2015-08-17 21:13:18,1:20
6,317,C41406,Central_to_Alborada,2015-08-17 21:16:11,2015-08-17 22:27:21,1:11


In [41]:
direction_Alborada_central = query[(query['Direction'] == 'Alborada_to_Central')]
direction_Alborada_central_sorted = direction_Alborada_central.sort(["Departure_Hour"])
direction_Alborada_central_sorted

,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time
13,317,C41408,Alborada_to_Central,2015-08-17 08:04:53,2015-08-17 10:04:00,1:59
7,317,C41401,Alborada_to_Central,2015-08-17 08:47:36,2015-08-17 10:33:19,1:45
1,317,C41405,Alborada_to_Central,2015-08-17 10:50:02,2015-08-17 14:22:03,3:32
15,317,C41408,Alborada_to_Central,2015-08-17 12:17:09,2015-08-17 13:44:24,1:27
9,317,C41401,Alborada_to_Central,2015-08-17 13:13:17,2015-08-17 14:48:01,1:34
3,317,C41406,Alborada_to_Central,2015-08-17 14:37:07,2015-08-17 16:44:22,2:07
17,317,C41408,Alborada_to_Central,2015-08-17 15:46:23,2015-08-17 17:44:18,1:57
11,317,C41401,Alborada_to_Central,2015-08-17 17:17:20,2015-08-17 19:35:08,2:17
5,317,C41406,Alborada_to_Central,2015-08-17 19:15:48,2015-08-17 21:07:17,1:51
